In [70]:
# mmt 기본 모델 (scene encoder 포함)에 대한 궤적 시각화

In [115]:
import argparse
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np


In [116]:
_path = os.getcwd()
print(f"현재 디렉토리: {_path}")

_path = _path.split("/")[:-2]
_path = "/".join(_path)
print(f"MMT4 디렉토리(_path):{_path}")

현재 디렉토리: /home/ngnadmin/dev/ngn_2024/MMT4/MMT/vis/tf
MMT4 디렉토리(_path):/home/ngnadmin/dev/ngn_2024/MMT4/MMT


In [117]:
sys.path.append(_path)

In [118]:
plt.style.use("seaborn-dark")
###### loader check ######
from mmt.data.loader_sdd import data_loader
###### model check #######
from mmt.models2_eth.mmtf_noscene_v2_2 import TrajectoryGenerator
# from mmt.models2_eth.mmtf_noscene_nostate import TrajectoryGenerator
from mmt.utils import (
    int_tuple,
    relative_to_abs,
    get_dset_path,
)
from mmt.losses import(
    displacement_error,
    final_displacement_error,
    l2_loss
)

In [128]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        
        self.num_samples = 20 # type=int

        # Dataset options
        self.dataset_name = 'sdd_dC'
        self.delim = '\t'
        self.loader_num_workers = 0 # 4 -> 1
        self.obs_len = 8
        self.pred_len = 8                   ############################### pred_len check !!!
        self.skip = 1
        # Optimization
        self.batch_size = 64                 ################################  batch_size check !!!
        self.num_iterations = 1000 # 
        self.num_epochs = 50 #                      
        # Model Options
        self.embedding_dim = 64
        self.num_lstm_layers = 1
        self.num_tf_layers = 4
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        self.state_type = 2
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        # self.neighborhood_size = 1024 # type=float
        # self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        # 궤적 이미지 출력 폴더
        self.output_dir = os.getcwd()+ '/fig/noscene/state0/obs8/2'
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        # self.checkpoint_name = 'checkpoint_basic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 0 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "0" # type=str   

        # Log
        # self.log_dir = "./"
        self.restore_path = "/home/ngnadmin/dev/ngn_2024/MMT4/MMT/scripts2_sdd/output/noscene_v2_2/state2/obs8/2/tf_2.2_2_with_model.pt"
        # self.restore_path = os.getcwd()[:-4]+ '/scripts/output/scene/state2/obs8/'+'scene_state_v2_with_model.pt'


args = CreateArg() 

In [129]:
print(f"restore_path: {args.restore_path}")
print(f"output_dir: {args.output_dir}")
print(f"batch_size: {args.batch_size}")

restore_path: /home/ngnadmin/dev/ngn_2024/MMT4/MMT/scripts2_sdd/output/noscene_v2_2/state2/obs8/2/tf_2.2_2_with_model.pt
output_dir: /home/ngnadmin/dev/ngn_2024/MMT4/MMT/vis/tf/fig/noscene/state0/obs8/2
batch_size: 64


In [122]:
def evaluate_helper(error, seq_start_end, model_output_traj, model_output_traj_best):
    error = torch.stack(error, dim=1)
    for (start, end) in seq_start_end:
        start = start.item()
        end = end.item()
        _error = error[start:end]
        _error = torch.sum(_error, dim=0)
        min_index = _error.min(0)[1].item()
        model_output_traj_best[:, start:end, :] = model_output_traj[min_index][
            :, start:end, :
        ]
    return model_output_traj_best

In [123]:
def get_generator(checkpoint):
    # n_units = (
    #     [args.traj_lstm_hidden_size]
    #     + [int(x) for x in args.hidden_units.strip().split(",")]
    #     + [args.graph_lstm_hidden_size]
    # )
    # n_heads = [int(x) for x in args.heads.strip().split(",")]
    generator = TrajectoryGenerator(
        obs_len=args.obs_len,
        pred_len=args.pred_len,
        state_type = args.state_type, # 
        embedding_dim=args.embedding_dim,
        encoder_h_dim=args.encoder_h_dim_g,
        decoder_h_dim=args.decoder_h_dim_g,
        mlp_dim=args.mlp_dim,
        num_lstm_layers=args.num_lstm_layers,
        num_tf_layers=args.num_tf_layers,
        noise_dim=args.noise_dim,
        noise_type=args.noise_type,
        noise_mix_type=args.noise_mix_type,
        pooling_type=args.pooling_type,
        pool_every_timestep=args.pool_every_timestep,
        dropout=args.dropout,
        bottleneck_dim=args.bottleneck_dim,
        # neighborhood_size=args.neighborhood_size,
        # grid_size=args.grid_size,
        batch_norm=args.batch_norm)
    generator.load_state_dict(checkpoint["g_state"])
    generator.cuda()
    generator.eval()
    return generator

In [124]:
def cal_ade_fde(pred_traj_gt, pred_traj_fake):
    ade = displacement_error(pred_traj_fake, pred_traj_gt, mode="raw")
    fde = final_displacement_error(pred_traj_fake[-1], pred_traj_gt[-1], mode="raw")
    return ade, fde

In [126]:
def plot_trajectory(args, loader, generator):
    ground_truth_input = []
    all_model_output_traj = []
    ground_truth_output = []
    pic_cnt = 0
    with torch.no_grad():
        for batch in loader:
            batch = [tensor.cuda() for tensor in batch]
            (obs_traj, obs_sel_state,    
            pred_traj_gt, _, 
            obs_traj_rel, pred_traj_gt_rel, 
            _, loss_mask, seq_start_end
            # ,  _
            ) = batch
            ade = []
            ground_truth_input.append(obs_traj)
            ground_truth_output.append(pred_traj_gt)
            model_output_traj = []
            model_output_traj_best = torch.ones_like(pred_traj_gt).cuda() # 동일한 shpae의 tensro를 1로 채워줌

            for _ in range(args.num_samples):

                pred_traj_fake_rel = generator(
                    obs_traj, obs_traj_rel, seq_start_end,
                    obs_sel_state
                )

                # pred_traj_fake_rel = pred_traj_fake_rel[-args.pred_len :] #

                pred_traj_fake = relative_to_abs(pred_traj_fake_rel, obs_traj[-1]) #

                model_output_traj.append(pred_traj_fake) # 

                ade_, fde_ = cal_ade_fde(pred_traj_gt, pred_traj_fake)
                ade.append(ade_)
            model_output_traj_best = evaluate_helper(
                ade, seq_start_end, model_output_traj, model_output_traj_best
            )
            all_model_output_traj.append(model_output_traj_best)

            for (start, end) in seq_start_end:
                plt.figure(figsize=(7.88,10.8), dpi=100)
                ground_truth_input_x_piccoor = (
                    obs_traj[:, start:end, :].cpu().numpy()[:, :, 0].T
                )
                ground_truth_input_y_piccoor = (
                    obs_traj[:, start:end, :].cpu().numpy()[:, :, 1].T
                )
                ground_truth_output_x_piccoor = (
                    pred_traj_gt[:, start:end, :].cpu().numpy()[:, :, 0].T
                )
                ground_truth_output_y_piccoor = (
                    pred_traj_gt[:, start:end, :].cpu().numpy()[:, :, 1].T
                )
                model_output_x_piccoor = (
                    model_output_traj_best[:, start:end, :].cpu().numpy()[:, :, 0].T
                )
                model_output_y_piccoor = (
                    model_output_traj_best[:, start:end, :].cpu().numpy()[:, :, 1].T
                )
                for i in range(ground_truth_output_x_piccoor.shape[0]):

                    # observed_line = plt.plot(
                    #     ground_truth_input_x_piccoor[i, :],
                    #     ground_truth_input_y_piccoor[i, :],
                    #     "r-",
                    #     linewidth=4,
                    #     label="Observed Trajectory",
                    # )[0]
                    # observed_line.axes.annotate(
                    #     "",
                    #     xytext=(
                    #         ground_truth_input_x_piccoor[i, -2],
                    #         ground_truth_input_y_piccoor[i, -2],
                    #     ),
                    #     xy=(
                    #         ground_truth_input_x_piccoor[i, -1],
                    #         ground_truth_input_y_piccoor[i, -1],
                    #     ),
                    #     # arrowprops=dict(
                    #     #     arrowstyle="->", color=observed_line.get_color(), lw=1
                    #     # ),
                    #     size=20,
                    # )
                    # ground_line = plt.plot(
                    #     np.append(
                    #         ground_truth_input_x_piccoor[i, -1],
                    #         ground_truth_output_x_piccoor[i, :],
                    #     ),
                    #     np.append(
                    #         ground_truth_input_y_piccoor[i, -1],
                    #         ground_truth_output_y_piccoor[i, :],
                    #     ),
                    #     "b-",
                    #     linewidth=4,
                    #     label="Ground Truth",
                    # )[0]
                    predict_line = plt.plot(
                        np.append(
                            ground_truth_input_x_piccoor[i, -1],
                            model_output_x_piccoor[i, :],
                        ),
                        np.append(
                            ground_truth_input_y_piccoor[i, -1],
                            model_output_y_piccoor[i, :],
                        ),
                        color="#FF00FF", # 핑크 "#ADFF2F", # 연두 "#ffff00", # 노랑
                        ls="--",
                        linewidth=4,
                        label="Predicted Trajectory",
                    )[0]

                plt.axis("on")
                
                plt.savefig(
                    args.output_dir+"/pic_{}.png".format(pic_cnt),
                    transparent=True
                )
                plt.close()
                pic_cnt += 1

In [113]:
def main(args):
    checkpoint = torch.load(args.restore_path) # .pt 파일을 불러올 경로
    generator = get_generator(checkpoint)
    # path = get_dset_path(args.dataset_name, args.dset_type)
    dst_path = args.output_dir
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
        print(f"Directory {dst_path} created.")
    else:
        print(f"Directory {dst_path} already exists.")
    path = '/home/ngnadmin/dev/ngn_2024/MMT4/MMT/mmt/datasets/sdd_dC/test2'
    _, loader = data_loader(args, path)
    plot_trajectory(args, loader, generator)

In [130]:
# 주요 변수 확인
print(f"불러올 모델 체크포인트 파일 경로: {args.restore_path}")
print(f"이미지가 저장될 위치: {args.output_dir}")
print("투명 배경 저장")
print("축on")

불러올 모델 체크포인트 파일 경로: /home/ngnadmin/dev/ngn_2024/MMT4/MMT/scripts2_sdd/output/noscene_v2_2/state2/obs8/2/tf_2.2_2_with_model.pt
이미지가 저장될 위치: /home/ngnadmin/dev/ngn_2024/MMT4/MMT/vis/tf/fig/noscene/state0/obs8/2
투명 배경 저장
축on


In [131]:
seed = 2024
deterministic = True

import random
import numpy as np
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False
	
main(args)

state_type:2번째 상태 인코더
state_size:1
input_dim:1152
Directory /home/ngnadmin/dev/ngn_2024/MMT4/MMT/vis/tf/fig/noscene/state0/obs8/2 created.
obs_len:8
pred_len:8
state_type:2
seq_len: 16
test2 / file_path:/home/ngnadmin/dev/ngn_2024/MMT4/MMT/mmt/datasets/sdd_dC/test2/sdd_test_v2.csv
1 of total 1 
len frames:452
num_sequences:437
e_count:0
